In [1]:
import torch
from torch.utils.data import DataLoader
from model_architecture import vulcanicModel
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning import Trainer
import wandb

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()

api_key = os.getenv("API_KEY")
wandb.login()

wandb: Currently logged in as: krzysztof-m-weber (kweber) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
data_train = torch.load("./data/train_dataset.pt", weights_only=False)
data_val = torch.load("./data/test_dataset.pt", weights_only=False)

In [4]:
batch_size = 64

train_loader = DataLoader(data_train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(data_val, batch_size=batch_size, shuffle=False)

In [5]:
wandb_logger = WandbLogger(
    project='vulcanology',
    log_model=True
)

early_stop_callback = EarlyStopping(
    monitor='val_loss',
    min_delta=0.00,
    patience=3,
    verbose=True,
    mode='min'
)

checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    save_top_k=1,
    mode="min",
    filename="epoch={epoch}-val_loss={val_loss:.4f}",
)

In [6]:
features = int(data_train[0][0].shape[0])
features

89

In [7]:
model = vulcanicModel(features=features)

In [8]:
trainer = Trainer(
    max_epochs=100,
    logger=wandb_logger,
    callbacks=[early_stop_callback, checkpoint_callback],
)
trainer.fit(model, train_loader, val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
wandb: WARNING The anonymous setting has no effect and will be removed in a future version.


┏━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name   ┃ Type   ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ dense1 │ Linear │ 23.0 K │ train │     0 │
│ 1 │ dense2 │ Linear │ 32.9 K │ train │     0 │
│ 2 │ dense3 │ Linear │  4.1 K │ train │     0 │
│ 3 │ dense4 │ Linear │     33 │ train │     0 │
└───┴────────┴────────┴────────┴───────┴───────┘

Trainable params: 60.1 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 60.1 K                                                                                               
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 4                                                                                           
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

c:\Users\krzys\Documents\studia\MLops\.venv\Lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

c:\Users\krzys\Documents\studia\MLops\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.p
y:434: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of 
the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.

...

c:\Users\krzys\Documents\studia\MLops\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.p
y:434: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value 
of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.

c:\Users\krzys\Documents\studia\MLops\.venv\Lib\site-packages\pytorch_lightning\loops\fit_loop.py:317: The number 
of training batches (9) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for 
log_every_n_steps if you want to see logs for the training epoch.

c:\Users\krzys\Documents\studia\MLops\.venv\Lib\site-packages\torch\nn\modules\loss.py:634: UserWarning: Using a 
target size (torch.Size([48])) that is different to the input size (torch.Size([48, 1])). This will likely lead to 
incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)

c:\Users\krzys\Documents\studia\MLops\.venv\Lib\site-packages\torch\nn\modules\loss.py:634: UserWarning: Using a 
target size (torch.Size([13])) that is different to the input size (torch.Size([13, 1])). This will likely lead to 
incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)

Metric val_loss improved. New best score: 8.827
Metric val_loss improved by 5.381 >= min_delta = 0.0. New best score: 3.445
Metric val_loss improved by 0.985 >= min_delta = 0.0. New best score: 2.460
Metric val_loss improved by 0.265 >= min_delta = 0.0. New best score: 2.195
Metric val_loss improved by 0.264 >= min_delta = 0.0. New best score: 1.931
Metric val_loss improved by 0.078 >= min_delta = 0.0. New best score: 1.853
Metric val_loss improved by 0.022 >= min_delta = 0.0. New best score: 1.830
Monitored metric val_loss did not improve in the last 3 records. Best score: 1.830. Signaling Trainer to stop.


In [21]:
wandb.finish()

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


epoch,▁▁▂▃▃▃▃▄▅▅▅▆▇▇▇▇█
train_loss,█▁
trainer/global_step,▁▁▂▃▃▃▃▄▅▅▅▆▆▇▇▇█
val_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,14
train_loss,1.57274
trainer/global_step,134
val_loss,1.61447
